In [ ]:
import pandas as pd
import sys
import os
sys.path.append('..')
from soysaucestock.stock_price_scraper import StockChartScraper
from soysaucestock.stock_chart_backtester import StockChartTester, TradeResult
from soysaucestock.technical_analyzer import TechnicalAnalyzer

In [ ]:
target_file = "C:\\temp\\TradeConfig\\TripleScreenBear.csv"
#target_file = "C:\\temp\\WorkingInProgress.csv"
df = pd.read_csv(target_file, parse_dates=True)
df = df.sort_index(ascending=False)

In [ ]:
trade_df = pd.DataFrame()

In [ ]:
ticker_list = df.Ticker.tolist()

In [ ]:
#get all the stock price
StockChartScraper.save_stocks(ticker_list)

In [ ]:
for i in range(0, len(ticker_list)):
    ticker = df.ix[i].Ticker.replace('/','.')
    start_date = df.ix[i].Date
    
    if not trade_df.empty and any(trade_df.EnterTicker == ticker) and trade_df.loc[trade_df.EnterTicker == ticker].ExitDate.max() > start_date:
        continue;
    
    print(ticker + ' ' + start_date)
    
    # start the trading
    trade = StockChartTester.process_macd_calc_short(ticker, start_date)
    
    # save trade
    trade_df = trade_df.append(trade.todf(), ignore_index=True)

In [ ]:
# save all the trades info for further documentation
# header = EnterDate,EnterDirection,EnterPrice,EnterTicker,ExitDate,ExitPrice,InitialStop,IsMissed,TargetPrice
result_file = "C:\\temp\\TradeConfig\\bear_trade_result.csv"
with open(result_file, 'a+') as f:
    trade_df.to_csv(f, header=False, index=False)

In [ ]:
d = StockChartTester()
for index in range(0, trade_df.EnterDate.size):
    trade = trade_df.iloc[index].copy()
    trade['EnterTicker'] = trade['EnterTicker'].replace('.','/').replace('/TO', '.TO')
    d.save_chart(trade, dest='TripleScreenBear')

In [ ]:
not trade_df.empty 

In [ ]:
# want to show any trade results
#d = StockChartTester()
#d.show_stock(a.enter_ticker, a.exit_date)